In [7]:
from preprocessing.preprocessing_label_encoding import  get_preprocessed_brfss_dataset_label_encoded_train_test_split, get_preprocessed_brfss_dataset_label_encoded_train_test_split_undersampled, get_preprocessed_brfss_dataset_label_encoded_train_test_split_oversampled

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, fbeta_score, recall_score, precision_score

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
data_train, data_validation, data_test, target_train, target_validation, target_test = get_preprocessed_brfss_dataset_label_encoded_train_test_split(include_test_data=True)
data_train.head(10)

,GenHealth,PhysHealth,MentHealth,Healthcare,MedCost,Checkup,HighBP,HighChol,HeartAttack,AngiCoro,...,Height,Weight,BMI,Education,Alcohol,Smoking,FruitCons,VegetCons,PhysActivity,Muscles
241445,3.0,0.0,0.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,...,0.855895,0.469484,4.0,3.0,1.0,3.0,2.0,1.0,3.0,2.0
107433,3.0,0.0,0.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,...,0.764192,0.392789,4.0,3.0,1.0,4.0,2.0,1.0,4.0,2.0
255533,2.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,...,0.855895,0.445989,4.0,3.0,1.0,4.0,1.0,2.0,3.0,1.0
31855,1.0,0.0,0.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,...,0.685590,0.203450,2.0,4.0,1.0,4.0,1.0,1.0,3.0,1.0
76133,2.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,...,0.698690,0.219079,2.0,4.0,1.0,4.0,1.0,1.0,1.0,1.0
93828,2.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,...,0.799127,0.289495,3.0,3.0,1.0,2.0,2.0,1.0,1.0,2.0
71236,2.0,0.0,0.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,...,0.742358,0.258203,3.0,2.0,1.0,4.0,2.0,1.0,1.0,2.0
212026,3.0,0.1,0.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,...,0.698690,0.234742,3.0,3.0,1.0,1.0,2.0,1.0,2.0,1.0
169431,3.0,1.0,0.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,...,0.777293,0.336450,4.0,4.0,1.0,3.0,2.0,2.0,3.0,2.0
60906,1.0,0.0,0.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,...,0.711790,0.211247,2.0,3.0,1.0,3.0,1.0,1.0,1.0,2.0


In [10]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(class_weight='balanced',y=target_train)

In [11]:
def validate_xgboost_classifier(estimator: XGBClassifier):
    estimator.fit(data_train, target_train, sample_weight=sample_weights)
    predictions = estimator.predict(data_validation)
    return accuracy_score(target_validation, predictions), fbeta_score(target_validation, predictions, beta=2), precision_score(target_validation, predictions), recall_score(target_validation, predictions)

In [18]:
min_child_weight_parameters = range(1, 20)
for param in min_child_weight_parameters:
    estimator = XGBClassifier(min_child_weight=param)
    accuracy, f_score, precision, recall = validate_xgboost_classifier(estimator)
    print(f"Min_child_weight_parameter {param} --> Accuracy: {accuracy}, F-Score: {f_score}, Precision: {precision}, Recall: {recall}")

Min_child_weight_parameter 1 --> Accuracy: 0.7507549650051515, F-Score: 0.6048936170212765, Precision: 0.3123489343001538, Recall: 0.7898319211001528
Min_child_weight_parameter 2 --> Accuracy: 0.7503286318257718, F-Score: 0.6026481607629427, Precision: 0.31144114411441143, Recall: 0.7864981247395472
Min_child_weight_parameter 3 --> Accuracy: 0.7499022986463921, F-Score: 0.6016133838491337, Precision: 0.3108435059936215, Recall: 0.78524795110432
Min_child_weight_parameter 4 --> Accuracy: 0.7495114932319608, F-Score: 0.6045578115708097, Precision: 0.3112970253718285, Recall: 0.7908042783719961
Min_child_weight_parameter 5 --> Accuracy: 0.749653604291754, F-Score: 0.6040161495962602, Precision: 0.31126806833114323, Recall: 0.789693012918461
Min_child_weight_parameter 6 --> Accuracy: 0.7487831740505205, F-Score: 0.6029852013843181, Precision: 0.3103316396219199, Recall: 0.7889984720100014
Min_child_weight_parameter 7 --> Accuracy: 0.7474686467474331, F-Score: 0.6045305387953848, Precision:

In [22]:
import numpy as np

subsample_parameters = np.arange(0.1, 1, 0.05)
for param in subsample_parameters:
    estimator = XGBClassifier(min_child_weight=10, subsample=param)
    accuracy, f_score, precision, recall = validate_xgboost_classifier(estimator)
    print(f"Subsample {param} --> Accuracy: {accuracy}, F-Score: {f_score}, Precision: {precision}, Recall: {recall}")

Subsample 0.1 --> Accuracy: 0.7345543041887235, F-Score: 0.5770849997895003, Precision: 0.2930518439337253, Recall: 0.7616335602166968
Subsample 0.15000000000000002 --> Accuracy: 0.7396347745763314, F-Score: 0.5885950482938967, Precision: 0.29975298034582754, Recall: 0.775385470204195
Subsample 0.20000000000000004 --> Accuracy: 0.7409137741144705, F-Score: 0.5915056122879567, Precision: 0.3014516129032258, Recall: 0.7788581747464925
Subsample 0.25000000000000006 --> Accuracy: 0.7429033289515756, F-Score: 0.5949211982929818, Precision: 0.3038083935699644, Recall: 0.7823308792887901
Subsample 0.30000000000000004 --> Accuracy: 0.742370412477351, F-Score: 0.5975167587166407, Precision: 0.3040982727175196, Recall: 0.7874704820113905
Subsample 0.3500000000000001 --> Accuracy: 0.7440579813123956, F-Score: 0.5953363380520112, Precision: 0.3047717055733088, Recall: 0.7816363383803306
Subsample 0.40000000000000013 --> Accuracy: 0.7424414680072476, F-Score: 0.5994102780117946, Precision: 0.304645

In [23]:
sampling_method = ["uniform", "gradient_based"]
for param in sampling_method:
    estimator = XGBClassifier(min_child_weight=10, sampling_method=param,)
    accuracy, f_score, precision, recall = validate_xgboost_classifier(estimator)
    print(f"Sampling method {param} --> Accuracy: {accuracy}, F-Score: {f_score}, Precision: {precision}, Recall: {recall}")

Sampling method uniform --> Accuracy: 0.748605535225779, F-Score: 0.6059931760866341, Precision: 0.3109673209722147, Recall: 0.7944158910959855
Sampling method gradient_based --> Accuracy: 0.748605535225779, F-Score: 0.6059931760866341, Precision: 0.3109673209722147, Recall: 0.7944158910959855


In [24]:
learning_rates = np.arange(0.1, 1, 0.05)
for param in learning_rates:
    estimator = XGBClassifier(min_child_weight=10, eta=param,)
    accuracy, f_score, precision, recall = validate_xgboost_classifier(estimator)
    print(f"Learning rate {param} --> Accuracy: {accuracy}, F-Score: {f_score}, Precision: {precision}, Recall: {recall}")

Learning rate0.1 --> Accuracy: 0.7408427185845738, F-Score: 0.6108257110637765, Precision: 0.30644316396018856, Recall: 0.8126128628976247
Learning rate0.15000000000000002 --> Accuracy: 0.7428500373041532, F-Score: 0.6106094571662023, Precision: 0.3078426194877211, Recall: 0.8096957910820948
Learning rate0.20000000000000004 --> Accuracy: 0.7463672860340356, F-Score: 0.6111661015521345, Precision: 0.31059024990635203, Recall: 0.8062230865397972
Learning rate0.25000000000000006 --> Accuracy: 0.7464738693288805, F-Score: 0.6057373413042101, Precision: 0.309281130345683, Recall: 0.796638422003056
Learning rate0.30000000000000004 --> Accuracy: 0.748605535225779, F-Score: 0.6059931760866341, Precision: 0.3109673209722147, Recall: 0.7944158910959855
Learning rate0.3500000000000001 --> Accuracy: 0.7509148399474189, F-Score: 0.6020205895517616, Precision: 0.3117377628166216, Recall: 0.7846923183775525
Learning rate0.40000000000000013 --> Accuracy: 0.7504885067680392, F-Score: 0.5983909174331506

In [26]:
min_split_loss_parameters = range(0,50)
best_param = [0,0]
for param in min_split_loss_parameters:
    estimator = XGBClassifier(min_child_weight=10, eta=0.2, gamma=param)
    accuracy, f_score, precision, recall = validate_xgboost_classifier(estimator)
    if f_score > best_param[0]:
        best_param[0] = f_score
        best_param[1] = param
    print(f"Min split loss {param} --> Accuracy: {accuracy}, F-Score: {f_score}, Precision: {precision}, Recall: {recall}")

print(f"Best min_split_loss: {best_param[1]} with f2-score {best_param[0]}")

Min split loss 0 --> Accuracy: 0.7463672860340356, F-Score: 0.6111661015521345, Precision: 0.31059024990635203, Recall: 0.8062230865397972
Min split loss 1 --> Accuracy: 0.7454080363804313, F-Score: 0.6092786326225608, Precision: 0.30939019827908715, Recall: 0.8041394638144187
Min split loss 2 --> Accuracy: 0.7444132589618787, F-Score: 0.6075267686223363, Precision: 0.3082012699429059, Recall: 0.8023336574524239
Min split loss 3 --> Accuracy: 0.7438981063701282, F-Score: 0.6082699499936967, Precision: 0.3080114905841047, Recall: 0.8042783719961105
Min split loss 4 --> Accuracy: 0.7446619533165169, F-Score: 0.6077853761178328, Precision: 0.308452133055689, Recall: 0.8024725656341158
Min split loss 5 --> Accuracy: 0.7452836892031123, F-Score: 0.6086315789473685, Precision: 0.3091317365269461, Recall: 0.8031671065425754
Min split loss 6 --> Accuracy: 0.7459587167371301, F-Score: 0.610313315926893, Precision: 0.310066324347454, Recall: 0.8052507292679539
Min split loss 7 --> Accuracy: 0.74

In [27]:
max_depth_parameters = range(1,50)
best_param = [0,0]
for param in max_depth_parameters:
    estimator = XGBClassifier(min_child_weight=10, eta=0.2, gamma=47, max_depth=param)
    accuracy, f_score, precision, recall = validate_xgboost_classifier(estimator)
    if f_score > best_param[0]:
        best_param[0] = f_score
        best_param[1] = param
    print(f"Max depth {param} --> Accuracy: {accuracy}, F-Score: {f_score}, Precision: {precision}, Recall: {recall}")

print(f"Max depth: {best_param[1]} with f2-score {best_param[0]}")

Max depth 1 --> Accuracy: 0.7418374960031264, F-Score: 0.6047253346762349, Precision: 0.30558795461774996, Recall: 0.8006667592721212
Max depth 2 --> Accuracy: 0.7392972608093225, F-Score: 0.6092450115719023, Precision: 0.30493086355335247, Recall: 0.8117794138074733
Max depth 3 --> Accuracy: 0.7336483461825417, F-Score: 0.6085230444761491, Precision: 0.30081259263044924, Recall: 0.8176135574385331
Max depth 4 --> Accuracy: 0.7348740540732582, F-Score: 0.6105661782964986, Precision: 0.302181482998771, Recall: 0.8196971801639117
Max depth 5 --> Accuracy: 0.7334174157103777, F-Score: 0.6100004130695197, Precision: 0.3010396493731526, Recall: 0.8205306292540631
Max depth 6 --> Accuracy: 0.7336661100650158, F-Score: 0.6129837445333773, Precision: 0.3019817073170732, Recall: 0.8255313237949715
Max depth 7 --> Accuracy: 0.7337549294773866, F-Score: 0.6093804907290654, Precision: 0.3011082171492774, Recall: 0.8190026392554521
Max depth 8 --> Accuracy: 0.7320318328773937, F-Score: 0.6085512746

In [28]:
data_train, data_validation, data_test, target_train, target_validation, target_test = get_preprocessed_brfss_dataset_label_encoded_train_test_split_undersampled(include_test_data=True)
estimator = XGBClassifier(min_child_weight=10, eta=0.2, gamma=47, max_depth=6)
estimator.fit(data_train, target_train)
predictions = estimator.predict(data_validation)
accuracy, f_score, precision, recall = accuracy_score(target_validation, predictions), fbeta_score(target_validation, predictions, beta=2), precision_score(target_validation, predictions), recall_score(target_validation, predictions)
print(f"Best model with undersampling --> Accuracy: {accuracy}, F-Score: {f_score}, Precision: {precision}, Recall: {recall}")

Best model with undersampling --> Accuracy: 0.7319252495825488, F-Score: 0.6079296971697336, Precision: 0.2994918699186992, Recall: 0.8187248228920684


In [29]:
data_train, data_validation, data_test, target_train, target_validation, target_test = get_preprocessed_brfss_dataset_label_encoded_train_test_split_oversampled(include_test_data=True)
estimator = XGBClassifier(min_child_weight=10, eta=0.2, gamma=47, max_depth=6)
estimator.fit(data_train, target_train)
predictions = estimator.predict(data_validation)
accuracy, f_score, precision, recall = accuracy_score(target_validation, predictions), fbeta_score(target_validation, predictions, beta=2), precision_score(target_validation, predictions), recall_score(target_validation, predictions)
print(f"Best model with oversampling --> Accuracy: {accuracy}, F-Score: {f_score}, Precision: {precision}, Recall: {recall}")

Best model with oversampling --> Accuracy: 0.7353892066650087, F-Score: 0.6107763351349114, Precision: 0.302590407796871, Recall: 0.8194193638005278


In [31]:
data_train, data_validation, data_test, target_train, target_validation, target_test = get_preprocessed_brfss_dataset_label_encoded_train_test_split(include_test_data=True)

estimator = XGBClassifier(min_child_weight=10, eta=0.2, gamma=47, max_depth=6)
estimator.fit(data_train, target_train, sample_weight=sample_weights)
predictions = estimator.predict(data_test)

accuracy, f_score, precision, recall = accuracy_score(target_test, predictions), fbeta_score(target_test, predictions, beta=2), precision_score(target_test, predictions), recall_score(target_test, predictions)
print(f"Model performance on test data --> Accuracy: {accuracy}, F-Score: {f_score}, Precision: {precision}, Recall: {recall}")

Model performance on test data --> Accuracy: 0.7344169109157119, F-Score: 0.6086731525564065, Precision: 0.30137337296300093, Recall: 0.8169190165300736
